# Building a Conversational Agent With Context Awareness
### Overview
This notebook outlines the process of conversational agent that maintains context across multiple interactions.We'll use a modern AI framework to build an agent capable of engaging in more natural and coherent conversations.

### Motivation
Many simple chatbots lack the ability to maintain context, leading to disjointed and frustrating user experiences. This project aims to solve the problem by implementing a conversational agent that can remember and refer to previous parts of the conversation, enhancing the overall interaction quality.

### Key Components
1. Language Model: The core AI component that generates responses.
2. Prompt Template: Defines the structure of our conversations.
3. History Manager: Manages conversation history and context.
4. Message Store: Stores the messages for each conversation session.

### Method Details
#### Setting up the environment
Begin by setting up the necessary AI framework and ensuring access to a suitable language model. This forms the foundation of our conversational agent.

#### Creating the Chat History Store
Implement a system to manage multiple conversation sessions. Each session should be uniquely identifiable and associated with its own message history.

#### Defining the Conversation Structure
Create a template that includes:
- A system message defining the AI's role
- A placeholder for conversation history
- The user's input
This structure guides the AI's responses and maintains consistency throughout the conversation.

#### Building the Conversational Chain
Combine the prompt template with the language model to create a basic conversational chain. Wrap this chain with a history management component that automatically handles the insertion and retrieval of conversation history.

#### Interacting with the Agent
To use the agent, invoke it with a user input and a session identifier. The history manager takes care of retrieving the appropriate conversation history, inserting it into the prompt, and storing new messages after each interaction.

### Conclusion
This approach to creating a conversational agent offers several advantages:

- Context Awareness: The agent can refer to previous parts of the conversation, leading to more natural interactions.
- Simplicity: The modular design keeps the implementation straightforward.
- Flexibility: It's easy to modify the conversation structure or switch to a different language model.
- Scalability: The session-based approach allows for managing multiple independent conversations.
With this foundation, you can further enhance the agent by:

- Implementing more sophisticated prompt engineering
- Integrating it with external knowledge bases
- Adding specialized capabilities for specific domains
- Incorporating error handling and conversation repair strategies
By focusing on context management, this conversational agent design significantly improves upon basic chatbot functionality, paving the way for more engaging and helpful AI assistants.

### Conversational Agent
This notebook demonstrates how to create a simple conversational agent using LangChain.

##### Import required libraries

In [1]:
!pip -q install langchain huggingface_hub openai  tiktoken python-dotenv
!pip -q install sentence_transformers langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/

In [3]:
!pip install -q langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.8 MB/s eta 0:00:00


In [1]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from langchain_groq import ChatGroq
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

##### Load environment variables and initalize the language model

In [2]:
llm = ChatGroq(model_name = "llama3-8b-8192", max_tokens=1000, temperature=0)

##### Create a simple in-memory store for chat histories

In [3]:
store = {}

def get_chat_history(session_id: str):
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

##### Create the prompt template

In [4]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

##### Combine the prompt and model into a runnable chain

In [5]:
chain = prompt | llm

##### Wrap the chain with message history

In [6]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

##### Example usage

In [7]:
session_id = "user_01"


response1 = chain_with_history.invoke(
    {"input": "Hello! How are u?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1.content)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2.content)

AI: Hello! I'm doing well, thank you for asking! I'm a helpful assistant, here to assist you with any questions or tasks you may have. How about you? How's your day going so far?
AI: Your previous message was "Hello! How are u?".


In [8]:
print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: Hello! How are u?
ai: Hello! I'm doing well, thank you for asking! I'm a helpful assistant, here to assist you with any questions or tasks you may have. How about you? How's your day going so far?
human: What was my previous message?
ai: Your previous message was "Hello! How are u?".
